In [2]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

# Read the Excel files
df = pd.read_excel("staff.xlsx")
df_departments = pd.read_excel("departments.xlsx")

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Filter out null values and create dropdown menu options from unique departments
department_options = [{'label': department, 'value': department} for department in df_departments['Department'].dropna().unique()]

# Add "Other" option for null department
department_options.append({'label': 'Other', 'value': 'Other'})

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    html.Div([
        html.Label('Select Project:'),
        dcc.Dropdown(
            id='project-dropdown',
            options=project_options,
            value=project_options[0]['value']  # Default value
        ),
    ]),
    html.Div([
        html.Label('Select Department:'),
        dcc.Dropdown(
            id='department-dropdown',
            options=department_options,
            value=department_options[0]['value']  # Default value
        ),
    ]),
    dcc.Graph(id='deployment-plan'),
])


# Define callback to update the deployment plan based on selected project and department
@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('project-dropdown', 'value'),
     Input('department-dropdown', 'value')]
)
def update_plan(selected_project, selected_department):
    # Filter the DataFrame based on the selected project and department
    filtered_data = df[(df['Project'] == selected_project) & (df['Job'].isin(df_departments[df_departments['Department'] == selected_department]['Job']))]

    # Get unique job titles for the selected project and department
    unique_jobs = filtered_data['Job'].unique()

    # Count staff for each job in each month
    staff_count = filtered_data.groupby(['Month', 'Job']).size().unstack().fillna(0)

    # Create data for the bar chart
    data = [
        go.Bar(
            x=staff_count.index,
            y=staff_count[job],
            name=job
        ) for job in unique_jobs
    ]

    # Define layout for the bar chart
    layout = go.Layout(
        title=f"Deployment Plan for {selected_project}",
        xaxis=dict(title='Months'),
        yaxis=dict(title='Number of Staff')
    )

    return {'data': data, 'layout': layout}


if __name__ == '__main__':
    app.run_server(debug=True)
